In [1]:
from utils.binaries import *
from utils.plotting import *

from utils.Auger.SD.UubRandoms import *

13:33:03 [INFO] utils          -- (  +410ms) -- import logging
13:33:03 [INFO] utils.binaries -- (   +78ms) -- import numpy as np
13:33:03 [INFO] utils.binaries -- (    +8ms) -- import uncertainties
13:33:03 [INFO] utils.binaries -- (  +170ms) -- import pandas as pd
13:33:03 [INFO] utils.binaries -- (    +2ms) -- import binaries.tools as tools
13:33:04 [INFO] utils.plotting -- (  +284ms) -- import matplotlib.pyplot as plt
13:33:04 [INFO] utils.plotting -- (  +368ms) -- import seaborn as so
13:33:04 [DBUG] utils.plotting -- (    +6ms) -- font size set to 9.5
13:33:04 [DBUG] utils.plotting -- (    +1ms) -- label size set to 13.0
13:33:04 [DBUG] utils.plotting -- (    +1ms) -- figure size set to [6.6, 3.3]
13:33:04 [DBUG] utils.plotting -- (    +1ms) -- markersize set to 2.0
13:33:04 [DBUG] utils.plotting -- (    +1ms) -- usetex set to False
13:33:04 [INFO] utils.plotting -- (    +2ms) -- import plotting.tools as plot


In [26]:
Traces = UubRandom('NuriaJr', 'wcd')

sum_tot = 0

for file in Traces:
    traces = file['traces']
    vem_peak = file['vem_peak']

    calibrated_traces = traces / vem_peak[:, :, np.newaxis]
    for trace in tools.ProgressBar(calibrated_traces, desc='reading traces in file', newline=False):

        # time_over_threshold(trace, multiplicity=12, threshold=0.001)
        # raise StopIteration

        sum_tot += time_over_threshold(trace, multiplicity=12, threshold=0.2)
        

KeyboardInterrupt: 

In [28]:
print(sum_tot)

44


In [25]:
def time_over_threshold(traces : np.ndarray, threshold : float = 0.2, multiplicity : int = 12) -> bool :
    
    pmt_multiplicity_check = lambda sums : sum(sums > multiplicity) > 1

    first_120_bins = traces[:, :120]
    pmt_running_sum = (first_120_bins >= threshold).sum(axis=1)

    for i in range(120, traces.shape[1] + 1):
        if pmt_multiplicity_check(pmt_running_sum) : return True            # check multiplicity for each window
        if i == traces.shape[1]: return False                               # we've reached the end of the trace

        new_over_threshold = np.array(traces[:, i] > threshold, dtype=int)
        old_over_threshold = np.array(traces[:, i - 120] > threshold, dtype=int)
        pmt_running_sum += new_over_threshold - old_over_threshold


    # windows = np.lib.stride_tricks.sliding_window_view(traces, (3, 120))[0]
    # for pmt1, pmt2, pmt3 in windows:

    #     pmt1_active = (pmt1 > threshold).sum() >= multiplicity
    #     pmt2_active = (pmt2 > threshold).sum() >= multiplicity
    #     pmt3_active = (pmt3 > threshold).sum() >= multiplicity

    #     if sum([pmt1_active, pmt2_active, pmt3_active]) > 1:
    #         return True
    # else: return False

In [5]:
windows = np.lib.stride_tricks.sliding_window_view(trace, (3, 120))[0]

In [23]:
pmt1, pmt2, pmt3 = windows[0]
threshold, multiplicity = 0.2, 12

pmt1_active = (pmt1 > threshold).sum() >= multiplicity
pmt2_active = (pmt2 > threshold).sum() >= multiplicity
pmt3_active = (pmt3 > threshold).sum() >= multiplicity
if sum([pmt1_active, pmt2_active, pmt3_active]) > 1:
    print(True)
else:
    print(False)

False


In [68]:
from itertools import product

multiplicities = range(5, 20)
threshold = np.linspace(0.01, .25, 25)
params = list(product(multiplicities, threshold))
print(len(params))

375


In [69]:
params[0]

(5, 0.01)

In [58]:
1.2 * 0.2

0.24

In [70]:
print(threshold)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25]
